# Source

> Summary of probable SARS cases with onset of illness from 1 November 2002 to 31 July 2003
> https://www.who.int/csr/sars/country/table2004_04_21/en/

# Libraries

In [1]:
# to get web contents
import requests 
# scrap and clean web contents
from bs4 import BeautifulSoup

# numerical opeations
import numpy as np
# storing and processing in a dataframe
import pandas as pd

# to hide warnings
import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
# link of the data
url = 'https://www.who.int/csr/sars/country/table2004_04_21/en/'

# headers
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

# get web contents
r = requests.get(url, headers=header)

# save data in a dataframe
df = pd.read_html(r.text)[0]

# first few rows
df

,Unnamed: 0,Cumulative number of cases,Cumulative number of cases.1,Cumulative number of cases.2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Areas,Female,Male,Total,Median age (range),Number of deaths^a,Case fatality ratio (%),Number of imported cases (%),Number of HCW affected (%),Date onset first probable case,Date onset last probable case
1,Australia,4,2,6,15 (1-45),0,0,6 (100),0 (0),26-Feb-03,1-Apr-03
2,Canada,151,100,251,49 (1-98),43,17,5 (2),109 (43),23-Feb-03,12-Jun-03
3,China,2674,2607,5327^b,Not available,349,7,Not Applicable,1002 (19),16-Nov-02,3-Jun-03
4,"China, Hong Kong Special Administrative Region",977,778,1755,40 (0-100),299,17,Not Applicable,386 (22),15-Feb-03,31-May-03
5,"China, Macao Special Administrative Region",0,1,1,28,0,0,1 (100),0 (0),5-May-03,5-May-03
6,"China, Taiwan",218,128,346^c,42 (0-93),37,11,21 (6),68 (20),25-Feb-03,15-Jun-03
7,France,1,6,7,49 (26 - 61),1,14,7 (100),2 (29)^d,21-Mar-03,3-May-03
8,Germany,4,5,9,44 (4-73),0,0,9 (100),1 (11),9-Mar-03,6-May-03
9,India,0,3,3,25 (25-30),0,0,3 (100),0 (0),25-Apr-03,6-May-03


# Preprocessing

In [3]:
# subselect rows
df = df.loc[1:29]

# rename columns
df.columns = ['Country/Region', 'Cumulative male cases', 'Cumulative female cases', 
              'Cumulative total cases', 'Median age (range)', 'No. of deaths', 
              'Case fatalities ratio (%)', 'Number of imported cases (%)', 
              'Number of HCW affected (%)', 'Date onset first probable case', 
              'Date onset last probable case']

df

,Country/Region,Cumulative male cases,Cumulative female cases,Cumulative total cases,Median age (range),No. of deaths,Case fatalities ratio (%),Number of imported cases (%),Number of HCW affected (%),Date onset first probable case,Date onset last probable case
1,Australia,4,2,6,15 (1-45),0,0,6 (100),0 (0),26-Feb-03,1-Apr-03
2,Canada,151,100,251,49 (1-98),43,17,5 (2),109 (43),23-Feb-03,12-Jun-03
3,China,2674,2607,5327^b,Not available,349,7,Not Applicable,1002 (19),16-Nov-02,3-Jun-03
4,"China, Hong Kong Special Administrative Region",977,778,1755,40 (0-100),299,17,Not Applicable,386 (22),15-Feb-03,31-May-03
5,"China, Macao Special Administrative Region",0,1,1,28,0,0,1 (100),0 (0),5-May-03,5-May-03
6,"China, Taiwan",218,128,346^c,42 (0-93),37,11,21 (6),68 (20),25-Feb-03,15-Jun-03
7,France,1,6,7,49 (26 - 61),1,14,7 (100),2 (29)^d,21-Mar-03,3-May-03
8,Germany,4,5,9,44 (4-73),0,0,9 (100),1 (11),9-Mar-03,6-May-03
9,India,0,3,3,25 (25-30),0,0,3 (100),0 (0),25-Apr-03,6-May-03
10,Indonesia,0,2,2,56 (47-65),0,0,2 (100),0 (0),6-Apr-03,17-Apr-03


In [4]:
# save raw data
df.to_csv('summary_data_raw.csv', index = False)

# Data Cleaning

In [5]:
# replace strings / characters
# ===========================

for col in df.columns:
    # replace special notes
    df[col] = df[col].str.replace('\^[abcde]', '')
    # replace with abbreviation
    df[col] = df[col].str.replace('China, ', '')
    df[col] = df[col].str.replace('Special Administrative Region', 'SAR, China')
    
# df

In [6]:
# extract just the numbers from the columns
# =========================================

cols = ['Cumulative male cases', 'Cumulative female cases', 'Cumulative total cases', 
        'No. of deaths', 'Case fatalities ratio (%)']

for col in cols:
    df[col] = df[col].str.extract('(\d*)')
    
# df

In [7]:
# extract number, range, percentage ...
df['Median age'] = df['Median age (range)'].str.extract('(\d*)')
df['Age range'] = df['Median age (range)'].str.extract('\((\d+\-\d+)\)')

df['Number of Imported cases'] =  df['Number of imported cases (%)'].str.extract('(\d*)')
df['Percentage of Imported cases'] = df['Number of imported cases (%)'].str.extract('\((\d+)\)')

df['Number of HCW affected'] =  df['Number of HCW affected (%)'].str.extract('(\d*)')
df['Percentage of HCW affected'] = df['Number of HCW affected (%)'].str.extract('\((\d+)\)')

# drop redundant columns
df = df.drop(['Median age (range)', 'Number of imported cases (%)', 'Number of HCW affected (%)'], axis=1)

In [8]:
# fix date time format
df['Date onset first probable case'] = pd.to_datetime(df['Date onset first probable case'])
df['Date onset last probable case'] = pd.to_datetime(df['Date onset last probable case'])

In [9]:
# fix empty string with nan
df = df.replace('', np.nan)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 1 to 29
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Country/Region                  29 non-null     object        
 1   Cumulative male cases           29 non-null     object        
 2   Cumulative female cases         29 non-null     object        
 3   Cumulative total cases          29 non-null     object        
 4   No. of deaths                   29 non-null     object        
 5   Case fatalities ratio (%)       29 non-null     object        
 6   Date onset first probable case  29 non-null     datetime64[ns]
 7   Date onset last probable case   29 non-null     datetime64[ns]
 8   Median age                      28 non-null     object        
 9   Age range                       18 non-null     object        
 10  Number of Imported cases        26 non-null     object        
 11  Percenta

In [11]:
df

,Country/Region,Cumulative male cases,Cumulative female cases,Cumulative total cases,No. of deaths,Case fatalities ratio (%),Date onset first probable case,Date onset last probable case,Median age,Age range,Number of Imported cases,Percentage of Imported cases,Number of HCW affected,Percentage of HCW affected
1,Australia,4,2,6,0,0,2003-02-26,2003-04-01,15,1-45,6,100,0,0
2,Canada,151,100,251,43,17,2003-02-23,2003-06-12,49,1-98,5,2,109,43
3,China,2674,2607,5327,349,7,2002-11-16,2003-06-03,NaN,NaN,NaN,NaN,1002,19
4,"Hong Kong SAR, China",977,778,1755,299,17,2003-02-15,2003-05-31,40,0-100,NaN,NaN,386,22
5,"Macao SAR, China",0,1,1,0,0,2003-05-05,2003-05-05,28,NaN,1,100,0,0
6,Taiwan,218,128,346,37,11,2003-02-25,2003-06-15,42,0-93,21,6,68,20
7,France,1,6,7,1,14,2003-03-21,2003-05-03,49,NaN,7,100,2,29
8,Germany,4,5,9,0,0,2003-03-09,2003-05-06,44,4-73,9,100,1,11
9,India,0,3,3,0,0,2003-04-25,2003-05-06,25,25-30,3,100,0,0
10,Indonesia,0,2,2,0,0,2003-04-06,2003-04-17,56,47-65,2,100,0,0


In [12]:
# save clean data
df.to_csv('summary_data_clean.csv', index = False)